# Detector de matriculas

In [6]:
import cv2
import pytesseract
import re
from ultralytics import YOLO
from pytesseract import Output
import easyocr
import math

### 1 modelo posible

In [ ]:

reader = easyocr.Reader(['en']) # Inicializamos el lector de easyocr

# Carga del modelo preentrenado YOLO (nano)
model = YOLO('yolov8n.pt')

# Etiqueta de las distintas clases relevantes 
classNames = ["person", "bicycle", "car", "motorbike", "bus", "truck"]

video_path = 'video.mp4'
vid = cv2.VideoCapture(video_path)

while True:
    ret, img = vid.read()
  
    # Verificamos si hay una imagen válida
    if ret:
        # Reducimos la resolución de la imagen para mejorar la velocidad
        img = cv2.resize(img, (640, 600))

        # Seguimiento en tiempo real para clases relevantes (personas y vehículos)
        results = model.track(img, persist=True, classes=[0, 1, 2, 3, 5, 7])

        # Iteramos sobre cada detección
        for r in results:
            boxes = r.boxes

            for box in boxes:
                # Extraemos las coordenadas de la caja delimitadora
                x1, y1, x2, y2 = map(int, box.xyxy[0])

                # Etiqueta de seguimiento
                if box.id is not None:
                    track_id = str(int(box.id[0].tolist()))
                else:
                    track_id = ''

                # Nivel de confianza
                confidence = math.ceil((box.conf[0] * 100)) / 100

                # Clase detectada
                cls = int(box.cls[0])
                class_name = classNames[cls] if cls < len(classNames) else "unknown"

                color = (0, 255, 0) if cls == 0 else (0, 0, 255)  # Verde para personas, rojo para vehículos

                # Dibujamos el contenedor y la etiqueta de clase
                cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
                cv2.putText(img, f"{track_id} {class_name} {confidence:.2f}", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

                # Si es una persona, tapamos la cara
                if cls == 0: 
                    face_height = int((y2 - y1) * 0.4)  # Asumimos que la cara ocupa el 40% superior del cuerpo
                    cv2.rectangle(img, (x1, y1), (x2, y1 + face_height), (0, 0, 0), -1)  # Rectángulo negro

                # Si es un vehículo, intentar detectar la matrícula
                if cls in [2, 3, 5, 7]: 
                    # Enfocamos en la parte baja del vehículo
                    plate_img = img[int(y1 + (y2 - y1) * 0.6):y2, x1:x2]

                    # Preprocesamos la imagen de la matrícula
                    gray_plate_img = cv2.cvtColor(plate_img, cv2.COLOR_BGR2GRAY)
                    enhanced_plate_img = cv2.equalizeHist(gray_plate_img)  # Aumentamos el contraste
                    _, binary_plate_img = cv2.threshold(enhanced_plate_img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

                    # Detectamos el texto en la imagen de la matrícula
                    plate_text = reader.readtext(binary_plate_img, detail=0)
                    plate_text_str = ' '.join([text for text in plate_text if any(char.isalnum() for char in text)])

                    # Mostramos el texto detectado en la imagen
                    cv2.putText(img, plate_text_str.strip(), (x1, y2 + 20),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)

        cv2.imshow('Detección y super seguimiento', img)
    else:
        break 

    # Salimos del bucle si se presiona ESC
    if cv2.waitKey(20) == 27:
        break

vid.release()
cv2.destroyAllWindows()

### 2 modelo posible

In [ ]:
# Tamaño del video
desired_size = (800, 500)

model = YOLO('modelo.pt')  # Modelo de yolo preentrenado

pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract'

def preprocess_plate(plate):
    """Preprocesar la imagen de la matrícula para mejorar la detección de texto."""
    gray = cv2.cvtColor(plate, cv2.COLOR_BGR2GRAY)
    # Aplicamos umbral adaptativo para resaltar caracteres
    processed_plate = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
    return processed_plate

def show_image(window_name, image):
    resized_image = cv2.resize(image, desired_size)
    cv2.imshow(window_name, resized_image)

# Leemos el video
video_path = "video.mp4"
cap = cv2.VideoCapture(video_path)

# Verificamos si el video se abrió correctamente
if not cap.isOpened():
    print("Error al abrir el video")
    exit()

while True:
    ret, frame = cap.read()

    # Si no se leen más cuadros, salimos
    if not ret:
        break

    # Detectamos matrículas con el modelo
    results = model(frame, verbose=False)

    # Iteramos sobre cada detección en el resultado
    for r in results:
        boxes = r.boxes

        for box in boxes:
            # Coordenadas del contenedor
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)  # Convertimos a valores enteros

            # Extraemos solo la matrícula
            plate = frame[y1:y2, x1:x2]

            # Preprocesar la imagen de la matrícula
            processed_plate = preprocess_plate(plate)

            # Recuadro de la matrícula
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)

            # Expresión regular para detectar la matrícula "0000 AAA"
            plate_pattern = re.compile(r'^\d{4}[A-Z]{3}$')

            # Utilizamos pytesseract para la detección de texto en la imagen preprocesada
            text = pytesseract.image_to_string(processed_plate, config='--psm 8', output_type=Output.STRING)

            # Limpiamos el texto detectado
            text = re.sub(r'[^A-Za-z0-9]', '', text).strip()

            # Verificamos si el texto cumple con el formato de matrícula
            if plate_pattern.match(text):
                print("Matrícula detectada:", text)
            else:
                print("Texto detectado (no coincide con patrón de matrícula):", text)

    # Mostramos el video
    show_image("Super mega detector", frame)

    # Salimos si se presiona la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar el video
cap.release()
cv2.destroyAllWindows()


### 3 posible modelo

In [ ]:

# Cargamos nuestro modelo YOLO preentrenado
model = YOLO('best.pt')

classNames = ["Super matricula"]  # Clase de matrícula

# Inicializa el lector de EasyOCR
reader = easyocr.Reader(['es'])

# Ruta del video
video_path = 'video.mp4'

# Abrimos
cap = cv2.VideoCapture(video_path)

while cap.isOpened():
    ret, img = cap.read()
    
    if not ret:
        break  # Salimos del bucle si se acaba el video

    results = model(img, stream=True)

    # Para cada detección
    for r in results:
        boxes = r.boxes
        for box in boxes:
            # Tomamos el contenedor
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            
            # Confianza
            confidence = math.ceil((box.conf[0] * 100)) / 100
            print("Confianza: ", confidence)

            # Clase
            cls = int(box.cls[0])
            print("Nombre de la clase: ", classNames[0])

            # Dibujamos el contenedor
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(img, f"{classNames[0]} {confidence}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            # Recortamos el área de interés (matrícula)
            img_cropped = img[y1:y2, x1:x2]
            if img_cropped.size > 0:
                # Reconocimiento de una imagen
                result = reader.readtext(img_cropped, allowlist='0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ')
                output = ""

                if len(result) >= 1:
                    first = result[0][1]
                else:
                    first = ""

                if len(result) >= 2:
                    second = result[1][1]
                else:
                    second = ""

                if first.isnumeric():
                    output = first + " " + second
                else:
                    output = second + " " + first

                print("La matrícula detectada es:", output)

    cv2.imshow('Detector increible de Matriculas', img)

    # Salimos del bucle si se presiona la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


### Otro posible modelo

In [ ]:

# Configuramos la ruta de Tesseract
pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract'

# Cargamos nuestro modelo YOLO
model = YOLO('best.pt')

classNames = ['Matricula'] 

# Captura desde la webcam
vid = cv2.VideoCapture("video.mp4")
  
while True:      
    # Leemos fotograma por fotograma
    ret, img = vid.read()
  
    # Si hay una imagen válida
    if ret:  
        # Realiza inferencia en el fotograma
        results = model(img, stream=True)
        
        # Para cada detección
        for r in results:
            boxes = r.boxes

            for box in boxes:
                # Extraemos las coordenadas del contenedor
                x1, y1, x2, y2 = box.xyxy[0]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)  # Conviertimos a enteros
                
                # Confianza
                confidence = math.ceil((box.conf[0] * 100)) / 100

                # Clase
                cls = int(box.cls[0])

                # Recortamos la región de interés (ROI)
                cropped = img[y1:y2, x1:x2]

                # Aplica reconocimiento de texto a la imagen recortada
                text = pytesseract.image_to_string(cropped)

                # Convierte el identificador numérico de clase a un color RGB
                escala = int((cls / len(classNames)) * 255 * 3)
                if escala >= 255 * 2:
                    R = 255
                    G = 255
                    B = escala - 255 * 2
                elif escala >= 255:
                    R = 255
                    G = escala - 255
                    B = 0
                else:
                    R = escala
                    G = 0
                    B = 0

                # Dibujamos el contenedor y el texto
                cv2.rectangle(img, (x1, y1), (x2, y2), (R, G, B), 3)
                cv2.putText(img, text.strip(), (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 255), 1)

        cv2.imshow('Video matriculas', img)
    
    # Detenemos el video si se presiona ESC
    if cv2.waitKey(20) == 27:
        break
  
vid.release()
cv2.destroyAllWindows()